In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## LangSmith

https://smith.langchain.com

In [2]:
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
# LANGCHAIN_API_KEY=<your-api-key>
# LANGCHAIN_PROJECT=<your-project>  # if not specified, defaults to "default"

In [3]:
#!pip install langsmith

In [4]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.tracers import LangChainTracer

llm = ChatOpenAI()
tracer = LangChainTracer(project_name="Napoleon v1")
llm.predict("How many brothers had Napoleon Bonaparte?", callbacks=[tracer])

'Napoleon Bonaparte had three brothers: Joseph Bonaparte, Lucien Bonaparte, and Louis Bonaparte.'

### See updates in Projects Area in LangSmith

## Basic LangSmith Operations

### Create new project with LangChainTracer

In [5]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="Churchill v1")
llm.predict("How old was Churchill when he was appointed PM?", callbacks=[tracer])

'Winston Churchill was 65 years old when he was appointed as the Prime Minister of the United Kingdom. He assumed office on May 10, 1940.'

### Check updates in Projects Area in LangSmith

### Alternative way to do the same

In [6]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="Cyrus v1"):
    llm.predict("When did Cyrus The Great reign in Persia?")

## Creating Tags in LangSmith Projects

In [7]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, tags=["History"])

prompt = PromptTemplate.from_template("Say {input}")

chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    tags=["Cyrus", "Persia"])

chain("When did the first Cyrus king reign in Persia?", tags=["Cyrus"])

{'input': 'When did the first Cyrus king reign in Persia?',
 'text': 'The first Cyrus king, Cyrus the Great, reigned in Persia from 559 BC to 530 BC.'}

### See that this went to default project since we did not set that differently

## Creating Groups in LangSmith Projects

In [8]:
from langchain.callbacks.manager import (
    trace_as_chain_group
)

with trace_as_chain_group("American History v1") as group_manager:
    pass

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

roman_llm = ChatOpenAI(temperature=0.9)

prompt = PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}?",
)

chain = LLMChain(
    llm=roman_llm, 
    prompt=prompt
)

with trace_as_chain_group("Roman History v1") as group_manager:
    chain.run(question="Who did Julius Caesar marry?", callbacks=group_manager)
    chain.run(question="Where did Julius Caesar fight?", callbacks=group_manager)
    chain.run(question="What was the name of the horse of Julius Caesar?", callbacks=group_manager)

## LangSmith Client

In [18]:
from langsmith import Client

client = Client()
project_runs = client.list_runs(project_name="default")
project_runs

<generator object Client.list_runs at 0x147102200>

In [24]:
from datetime import datetime, timedelta

todays_runs = client.list_runs(
    project_name="default",
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm",
)
todays_runs

<generator object Client.list_runs at 0x1466f9440>

In [21]:
# for run in todays_runs:
#     print(run)

In [22]:
# todays_runs = client.list_runs(
#     project_name="Churchill v1",
#     start_time=datetime.now() - timedelta(days=1),
#     run_type="llm",
# )

# for run in todays_runs:
#     print(run)

## Adding metadata to filter runs

In [13]:
chat_model = ChatOpenAI()
chain = LLMChain.from_string(
    llm=chat_model, 
    template="What's the answer to {input}?")

chain(
    {"input": "Who was the companion of Don Quixote?"}, 
    metadata={"source": "Cervantes"}
)

{'input': 'Who was the companion of Don Quixote?',
 'text': 'The main companion of Don Quixote in the novel "Don Quixote" by Miguel de Cervantes is Sancho Panza.'}

In [25]:
runs = list(client.list_runs(
    project_name="default",
    filter='has(metadata, \'{"source": "Cervantes"}\')',
))

#print(list(runs))

## Evaluating LLM App with Test Dataset in LangSmith

In [15]:
from langsmith import Client

example_inputs = [
  ("What is the largest mammal?", "The blue whale"),
  ("What do mammals and birds have in common?", "They are both warm-blooded"),
  ("What are reptiles known for?", "Having scales"),
  ("What's the main characteristic of amphibians?", "They live both in water and on land"),
]

client = Client()

dataset_name = "Elementary Animal Questions v1"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name, 
    description="Questions and answers about animal phylogenetics.",
)

for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"question": input_prompt},
        outputs={"answer": output_answer},
        dataset_id=dataset.id,
    )

In [16]:
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",
        "context_qa",
        "cot_qa",
    ]
)

In [17]:
client = Client()
llm = ChatOpenAI()
run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=llm,
    client=client,
    evaluation=evaluation_config,
    project_name="evalproject v1",
)

View the evaluation results for project 'evalproject v1' at:
https://smith.langchain.com/o/ec6a7494-139b-5170-8044-143bc78622a9/projects/p/72b8d49b-782d-44dd-81be-6a6970762986?eval=true

View all tests for Dataset Elementary Animal Questions v1 at:
https://smith.langchain.com/datasets/fd0aa228-8310-4082-b949-776429b7eac3
[------------------------------------------------->] 4/4

{'project_name': 'evalproject v1',
 'results': {'303e45d0-2324-48ca-af44-bcf9884cdd32': {'output': 'The main characteristic of amphibians is their ability to live both in water and on land. They have a dual life cycle, starting as aquatic larvae (such as tadpoles) and then transforming into terrestrial adults. Amphibians also have moist, permeable skin, which allows them to breathe through their skin. They typically lay their eggs in water and undergo metamorphosis during their development.',
   'input': {'question': "What's the main characteristic of amphibians?"},
   'feedback': [EvaluationResult(key='correctness', score=1, value='CORRECT', comment='CORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('b46ee973-6887-4450-94a9-af85613ea29d'))}, source_run_id=None, target_run_id=None),
    EvaluationResult(key='Contextual Accuracy', score=1, value='CORRECT', comment='CORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('910e6d60-29ee-4f53-aaa5-ebd5